In [75]:
# %%writefile generate_training_data.py
from skimpy import skim
import pandas as pd
import os
import xml.etree.ElementTree as ET

# Change the current working directory to the script's directory
# os.chdir(os.path.dirname(os.path.abspath(__file__)))

train1_data = r".\TrainingDatasets\TREC-2017-LiveQA-Medical-Train-1.xml"
train2_data = r".\TrainingDatasets\TREC-2017-LiveQA-Medical-Train-2.xml"

# Open the xml files and read the data
with open(train1_data, 'r', encoding='utf-8') as file:
    train1_data = file.read()

with open(train2_data, 'r', encoding='utf-8') as file:
    train2_data = file.read()

def parse_xml_training_data(xml_training_data):
    root = ET.fromstring(xml_training_data)

    # Extract data
    questions_data = []
    for question in root.findall('NLM-QUESTION'):
        question_id = question.get('questionid')
        f_ref = question.get('fRef')
        subject = question.find('SUBJECT').text if question.find('SUBJECT') is not None else ""
        question_text = question.find('MESSAGE').text if question.find('MESSAGE') is not None else ""
        
        for sub_question in question.find('SUB-QUESTIONS').findall('SUB-QUESTION'):
            sub_question_id = sub_question.get('subqid')
            focus = sub_question.find('ANNOTATIONS').find('FOCUS').text
            q_type = sub_question.find('ANNOTATIONS').find('TYPE').text
            
            for answer in sub_question.find('ANSWERS').findall('ANSWER'):
                answer_id = answer.get('answerid')
                pair_id = answer.get('pairid')
                answer_text = answer.text
                
                questions_data.append({
                    'question_id': question_id,
                    'f_ref': f_ref,
                    'subject': subject,
                    'question': question_text,
                    'sub_question_id': sub_question_id,
                    'focus': focus,
                    'type': q_type,
                    'answer_id': answer_id,
                    'pair_id': pair_id,
                    'answer_text': answer_text
                })

    return pd.DataFrame(questions_data)


df_train1 = parse_xml_training_data(train1_data)
df_train2 = parse_xml_training_data(train2_data)
# Join the two training datasets
df_train = pd.concat([df_train1, df_train2], ignore_index=True)
# Save the data to csv
df_train1.to_csv(r".\TrainingDatasets\TREC-2017-LiveQA-Medical-Train-1.csv", index=False)
df_train2.to_csv(r".\TrainingDatasets\TREC-2017-LiveQA-Medical-Train-2.csv", index=False)
df_train.to_csv(r".\TrainingDatasets\TREC-2017-LiveQA-Medical-Train.csv", index=False)
print(df_train1.shape)
print(df_train2.shape)
print(df_train.shape)

(388, 10)
(246, 10)
(634, 10)


In [82]:
skim(df_train)
display(df_train.head())

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 634    │ │ string      │ 10    │                                                          │
│ │ Number of columns │ 10     │ └─────────────┴───────┘                                                          │
│ └───────────────────┴────────┘                                                                                  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                    ┃ NA      ┃ NA %        ┃ words per row             ┃ total words           ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ question_id                    │     246 │        38.8 │                      0.61 │                   388 │  │
│ │ f_ref                          │     246 │        38.8 │                      0.61 │                   388 │  │
│ │ subject                        │     170 │       26.81 │                       2.4 │                  1551 │  │
│ │ question                       │       2 │        0.32 │                        47 │                 29592 │  │
│ │ sub_question_id                │     246 │        38.8 │                      0.61 │                   388 │  │
│ │ focus                          │       0 │           0 │                       1.9 │                  1233 │  │
│ │ type                           │       0 │           0 │                         1 │                   645 │  │
│ │ answer_id                      │     246 │        38.8 │                      0.61 │                   388 │  │
│ │ pair_id                        │     246 │        38.8 │                      0.61 │                   388 │  │
│ │ answer_text                    │       0 │           0 │                       190 │                121231 │  │
│ └────────────────────────────────┴─────────┴─────────────┴───────────────────────────┴───────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

,question_id,f_ref,subject,question,sub_question_id,focus,type,answer_id,pair_id,answer_text
0,Q1,11373,None,Literature on Cardiac amyloidosis. Please let...,Q1-S1,cardiac amyloidosis,information,Q1-S1-A1,1,Cardiac amyloidosis is a disorder caused by de...
1,Q1,11373,None,Literature on Cardiac amyloidosis. Please let...,Q1-S1,cardiac amyloidosis,information,Q1-S1-A2,2,"The term ""amyloidosis"" refers not to a single ..."
2,Q2,1-136434885,treatment options versus migraine types,Migraine seems to be a spectrum of conditions ...,Q2-S1,migraine,treatment,Q2-S1-A1,3,There is no specific cure for migraine headach...
3,Q2,1-136434885,treatment options versus migraine types,Migraine seems to be a spectrum of conditions ...,Q2-S1,migraine,treatment,Q2-S1-A2,4,There is no absolute cure for migraine since i...
4,Q3,1-123453375,None,DO I USE PYRIDOXINE TABLETS EVEN IF IM PREGNANT?,Q3-S1,pyridoxine,contraindication,Q3-S1-A1,5,"Before taking pyridoxine, tell your doc..."


In [90]:
columns_to_drop = ['question_id', 'f_ref', 'sub_question_id', 'answer_id', 'pair_id']
df_train_clean = df_train.drop(columns=columns_to_drop, inplace=False)
df_train_clean.head()
df_train_clean['subject'].isna().value_counts()

subject
False    464
True     170
Name: count, dtype: int64